# **Morphology**

In [ ]:
# Step 1: Upload CSV
from google.colab import files
uploaded = files.upload()

# Step 2: Load DataFrame
import pandas as pd
import numpy as np

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

# Step 3: Oversampling with noise
from collections import Counter

def oversample_with_noise(X, y, target_count=1000, noise_level=0.02):
    X_new, y_new = list(X), list(y)
    class_counts = Counter(y)

    for label in np.unique(y):
        current_count = class_counts[label]
        if current_count >= target_count:
            continue

        class_indices = np.where(y == label)[0]
        needed = target_count - current_count

        for _ in range(needed):
            idx = np.random.choice(class_indices)
            noisy_sample = X[idx] + np.random.normal(0, noise_level, X.shape[1])
            X_new.append(noisy_sample)
            y_new.append(label)

    return np.array(X_new), np.array(y_new)

# Step 4: Prepare features and labels
X = df.drop('label', axis=1).values
y = df['label'].values

# Step 5: Oversample
X_bal, y_bal = oversample_with_noise(X, y, target_count=1000, noise_level=0.02)

# Step 6: Train/Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.2, stratify=y_bal, random_state=42)

# Step 7: Normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 8: Train Random Forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 9: Evaluate
print("✅ Accuracy:", model.score(X_test, y_test))
print("\n🧾 Classification Report:\n", classification_report(y_test, model.predict(X_test)))




Saving coin_features_morphology.csv to coin_features_morphology.csv
✅ Accuracy: 0.5034891835310538

🧾 Classification Report:
               precision    recall  f1-score   support

    10c_back       0.65      0.78      0.71       200
   10c_front       0.53      0.47      0.49       200
    20c_back       0.54      0.41      0.47       200
   20c_front       0.49      0.36      0.42       200
    50c_back       0.46      0.51      0.48       200
   50c_front       0.52      0.51      0.51       200
     5c_back       0.51      0.53      0.52       200
    5c_front       0.50      0.35      0.41       200
     R1_back       0.44      0.72      0.54       200
    R1_front       0.54      0.38      0.44       200
     R2_back       0.53      0.45      0.48       200
    R2_front       0.48      0.54      0.51       200
     R5_back       0.56      0.34      0.43       200
    R5_front       0.44      0.65      0.53       266

    accuracy                           0.50      2866
   macro

In [ ]:
# Step 10: Convert to TensorFlow (via Keras Functional API)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# One-hot encode the labels
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_bal)
num_classes = len(label_encoder.classes_)

# Train/test split again with encoded labels
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X_bal, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# Normalize
X_train_nn = scaler.fit_transform(X_train_nn)
X_test_nn = scaler.transform(X_test_nn)

# Convert labels to categorical
y_train_nn_cat = to_categorical(y_train_nn, num_classes)
y_test_nn_cat = to_categorical(y_test_nn, num_classes)

# Define a small neural network
model_nn = keras.Sequential([
    layers.Input(shape=(X_train_nn.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_nn.fit(X_train_nn, y_train_nn_cat, epochs=20, batch_size=32, validation_data=(X_test_nn, y_test_nn_cat))

# Step 11: Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_nn)
tflite_model = converter.convert()

# Save the TFLite model
with open('coin_classifier_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Step 12: Download the TFLite model
files.download('coin_classifier_model.tflite')

Epoch 1/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.1440 - loss: 2.5683 - val_accuracy: 0.1950 - val_loss: 2.3792
Epoch 2/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2327 - loss: 2.2964 - val_accuracy: 0.2303 - val_loss: 2.2831
Epoch 3/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2646 - loss: 2.1782 - val_accuracy: 0.2446 - val_loss: 2.2255
Epoch 4/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2921 - loss: 2.0683 - val_accuracy: 0.2579 - val_loss: 2.1666
Epoch 5/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3202 - loss: 1.9996 - val_accuracy: 0.2851 - val_loss: 2.1161
Epoch 6/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3243 - loss: 1.9406 - val_accuracy: 0.2847 - val_loss: 2.0871
Epoch 7/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3565 - loss: 1.8777 - val_accuracy: 0.2917 - val_loss: 2.0795
Epoch 8/20
359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3607 - loss: 1.8468 - val_accuracy: 0.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**SMOTE**

In [ ]:
# Step 1: Upload CSV
from google.colab import files
uploaded = files.upload()

# Step 2: Load CSV
import pandas as pd

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

# Step 3: Prepare features and labels
X = df.drop('label', axis=1).values
y = df['label'].values

# Step 4: Apply SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Step 5: Train/Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)

# Step 6: Normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 7: Train Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

print("✅ Accuracy:", rf_model.score(X_test, y_test))
print("\n🧾 Classification Report:\n", classification_report(y_test, rf_model.predict(X_test)))


Saving coin_features_morphology.csv to coin_features_morphology (1).csv
✅ Accuracy: 0.764183920408712

🧾 Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.96      0.93       265
           1       0.79      0.79      0.79       266
           2       0.80      0.86      0.83       266
           3       0.71      0.72      0.71       266
           4       0.80      0.80      0.80       266
           5       0.74      0.72      0.73       265
           6       0.75      0.82      0.78       266
           7       0.75      0.70      0.73       266
           8       0.78      0.96      0.86       266
           9       0.78      0.75      0.76       265
          10       0.80      0.79      0.80       266
          11       0.71      0.68      0.69       265
          12       0.73      0.73      0.73       265
          13       0.60      0.43      0.50       266

    accuracy                           0.76      3719
   ma